In [5]:
import tqdm
import pandas as pd
from collections import Counter
from unicodedata import normalize

BIN_SIZE = 10

def get_ngrams(string: str, _sizes = (3, 4, 5), prefix="$"):
    ret = {}
    #string = normalize("NFC", string)
    for S in _sizes:
        nb = len(string) - (S-1)
        ret.update({
            prefix+k.replace(" ", "_"): v/nb
            for k, v in Counter([string[i:i+S] for i in range(len(string)-S+1)]).items()
        })
    return ret


df = pd.read_csv("cer.csv.gzip", compression="gzip", index_col=0)

In [6]:
print(df.shape)
df.drop_duplicates(subset=["manuscript", "page_id", "line_id", "transcription"], inplace=True)
print(df.shape)
df.head()

(322903, 7)
(300269, 7)


,model,lang,manuscript,page_id,line_id,transcription,CER
0,data-lat_only_3.mlmodel,lat,SBB_PK_Hdschr25,0,0,a ⁊utl,82.352942
1,data-lat_only_3.mlmodel,lat,SBB_PK_Hdschr25,0,1,t ut t̃ ps p,68.181819
2,data-lat_only_3.mlmodel,lat,SBB_PK_Hdschr25,0,2,tas t̃,78.260869
3,data-lat_only_3.mlmodel,lat,SBB_PK_Hdschr25,0,3,e ult ut p̃ t,68.000001
4,data-lat_only_3.mlmodel,lat,SBB_PK_Hdschr25,0,4,ba tt ss ttta,70.967740


# Compute Bins

In [7]:
df["CER_BINS"] = ""
df.loc[df.CER < 10, "CER_BINS"] = "Good"
df.loc[df.CER.between(10, 25, inclusive="left"), "CER_BINS"] = "Acceptable"
df.loc[df.CER.between(25, 50, inclusive="left"), "CER_BINS"] = "Bad"
df.loc[df.CER >= 50, "CER_BINS"] = "Very bad"
df.groupby("CER_BINS").size()

CER_BINS
Acceptable    54110
Bad           77058
Good          77210
Very bad      91891
dtype: int64

In [8]:
df['CER_BINS'].unique()

array(['Very bad', 'Bad', 'Acceptable', 'Good'], dtype=object)

# Compute ngrams

new_df = []

total = Counter()


for idx, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    if len(str(row.transcription)) > 15:
        cnt = get_ngrams(str(row.transcription).lower())
        total += cnt
        new_df.append({
            "idx": idx,
            "lang": row.lang,
            "bin": row.CER_BINS,
            "manuscript": row.manuscript,
            "page_id": row.page_id,
            "line_id": row.line_id,
            "transcription": row.transcription, 
            "CER": row.CER,
            **cnt
        })
    
del df  # For memory sake

In [14]:
def ngrams(row):
    return get_ngrams(str(row[1].transcription).lower())
from multiprocessing import Pool

all_ngrams = []
with Pool(processes=4) as pool:
    # print same numbers in arbitrary order
    for results in tqdm.tqdm(pool.imap_unordered(ngrams, df.iterrows()), total=len(df)):
        all_ngrams.append(results)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300269/300269 [00:31<00:00, 9415.43it/s]


{'$u_n': 0.03333333333333333,
 '$_ne': 0.03333333333333333,
 '$ne_': 0.03333333333333333,
 '$e_c': 0.03333333333333333,
 '$_co': 0.03333333333333333,
 '$con': 0.03333333333333333,
 '$ont': 0.03333333333333333,
 '$ntr': 0.03333333333333333,
 '$tre': 0.03333333333333333,
 '$ree': 0.03333333333333333,
 '$ee_': 0.03333333333333333,
 '$e_q': 0.03333333333333333,
 '$_qͥ': 0.03333333333333333,
 '$qͥ_': 0.03333333333333333,
 '$ͥ_m': 0.03333333333333333,
 '$_ml': 0.03333333333333333,
 '$ml̃': 0.03333333333333333,
 '$l̃t': 0.03333333333333333,
 '$̃t_': 0.03333333333333333,
 '$t_e': 0.03333333333333333,
 '$_es': 0.03333333333333333,
 '$est': 0.03333333333333333,
 '$st_': 0.03333333333333333,
 '$t_p': 0.03333333333333333,
 '$_po': 0.03333333333333333,
 '$pos': 0.03333333333333333,
 '$ost': 0.03333333333333333,
 '$ste': 0.03333333333333333,
 '$tei': 0.03333333333333333,
 '$eis': 0.03333333333333333,
 '$u_ne': 0.034482758620689655,
 '$_ne_': 0.034482758620689655,
 '$ne_c': 0.034482758620689655,
 '$e

## Cut ngrams appearing in less than .5% of the lines

In [17]:
total_presence = Counter([
    key
    for row in tqdm.tqdm(all_ngrams)
    for key in row 
    if key.startswith("$")
])
print(len(total_presence))
#filter_value = int(len(total_presence)/100)
#max_value = total_presence.most_common(filter_values)[-1][1]
#print(f"1% {filter_value}")

most_common = [key for key, _ in total_presence.most_common(2000)]
total_presence = {
    key: value
    for key, value in tqdm.tqdm(total_presence.items())
    if key in most_common
}
print(most_common)
#del total

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300269/300269 [00:01<00:00, 230843.35it/s]


1183787


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1183787/1183787 [00:17<00:00, 69463.55it/s]

['$_de', '$nt_', '$es_', '$ent', '$it_', '$_le', '$le_', '$_se', '$et_', '$est', '$re_', '$ne_', '$e_s', '$_es', '$de_', '$te_', '$st_', '$e_d', '$t_a', '$_⁊_', '$_en', '$_si', '$ant', '$t_e', '$e_p', '$_po', '$e_e', '$e_l', '$en_', '$_la', '$ent_', '$t_d', '$_au', '$is_', '$_ne', '$us_', '$_qu', '$_co', '$_de_', '$t_l', '$e_a', '$ie_', '$ut_', '$s_e', '$t_s', '$les', '$se_', '$_me', '$s_a', '$la_', '$_ce', '$_e_', '$_pe', '$_pa', '$ẽ_', '$t_p', '$res', '$s_s', '$s_d', '$_no', '$ns_', '$_li', '$il_', '$_est', '$tre', '$e_c', '$ere', '$oit', '$ont', '$_ma', '$ue_', '$s_p', '$_et', '$_a_', '$_re', '$_sa', '$_so', '$_te', '$_di', '$ien', '$e_m', '$ũ_', '$er_', '$e_t', '$e_n', '$si_', '$li_', '$_mo', '$ant_', '$e_de', '$or_', '$on_', '$t_i', '$_do', '$_s_', '$les_', '$_le_', '$ene', '$_al', '$ist', '$_to', '$ren', '$ui_', '$at_', '$t_m', '$ier', '$des', '$s_l', '$est_', '$nte', '$e_q', '$oit_', '$_su', '$e_f', '$t_c', '$ens', '$ele', '$s_c', '$̃t_', '$ur_', '$ce_', '$_et_', '$_lo', '$̃_s

# Sentences

In [12]:
import numpy as np
import random

random.shuffle(new_df)
KEPT = ["idx", "bin", "lang", "transcription", "manuscript", "page_id", "line_id", "CER", *total_presence.keys()]
# Optim
DTYPES = {
    "idx": np.int32,
    "page_id": np.int16,
    "line_id": np.int16,
    "CER": np.float16,
    **{
        feat: np.float16
        for feat in total_presence
    }
}

df = pd.DataFrame({
    key: pd.Series([
        row.get(key, np.nan)
        for row in new_df
    ], dtype=DTYPES.get(key))
    for key in KEPT})#, dtype={feature: np.float32 for feature in total_presence})
df.head()

,idx,bin,lang,transcription,manuscript,page_id,line_id,CER,$_q̃,$q̃_,...,$cie,$iez,$gie,$loi,$uei,$aen,$nz_,$esc,$inz,$r_⁊
0,184114,Good,fro,diuine chose.Apres ce bien lespace de.u.,bnf_fr_412_wauchier,50,44,4.878906,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,81772,Very bad,fro,t ts rs si t aẽl,bnf__arsenal3516_imagedumonde,6,163,58.625000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,348290,Good,fro,signor ⁊agemir ⁊aplorer.por ce qe tant,bnf_fr_412_wauchier,38,36,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.055542
3,51822,Acceptable,lat,¶louatibi due.qui natus,SBB_PK_Hdschr25,7,1,23.078125,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,52653,Acceptable,lat,ẽ ĩsinc malitie ⁊dic. Et cũẽ uũd aut spum...,CLM13027,0,36,17.734375,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df.dtypes

idx                int32
bin               object
lang              object
transcription     object
manuscript        object
                  ...   
$aen             float16
$nz_             float16
$esc             float16
$inz             float16
$r_⁊             float16
Length: 1008, dtype: object

In [14]:
df.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265025 entries, 0 to 265024
Columns: 1008 entries, idx to $r_⁊
dtypes: float16(1001), int16(2), int32(1), object(4)
memory usage: 516.1+ MB


In [15]:
df.to_hdf("features.hdf5", key='df')
df.head()

,idx,bin,lang,transcription,manuscript,page_id,line_id,CER,$_q̃,$q̃_,...,$cie,$iez,$gie,$loi,$uei,$aen,$nz_,$esc,$inz,$r_⁊
0,184114,Good,fro,diuine chose.Apres ce bien lespace de.u.,bnf_fr_412_wauchier,50,44,4.878906,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,81772,Very bad,fro,t ts rs si t aẽl,bnf__arsenal3516_imagedumonde,6,163,58.625000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,348290,Good,fro,signor ⁊agemir ⁊aplorer.por ce qe tant,bnf_fr_412_wauchier,38,36,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.055542
3,51822,Acceptable,lat,¶louatibi due.qui natus,SBB_PK_Hdschr25,7,1,23.078125,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,52653,Acceptable,lat,ẽ ĩsinc malitie ⁊dic. Et cũẽ uũd aut spum...,CLM13027,0,36,17.734375,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df["$abi"].value_counts()
df

In [17]:
df.bin.unique()

array(['Good', 'Very bad', 'Acceptable', 'Bad'], dtype=object)

In [13]:
df["idx 	bin 	transcription 	manuscript 	page_id 	line_id 	CER".split()].to_hdf("texts.hdf5", key='df')